In [1]:
import pathlib

import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.python.keras.models import Sequential
print(tf.__version__)


1.4.0


Using TensorFlow backend.


In [14]:
column_names = ['/render/frame time/rendering/microseconds','ar_GI_diffuse_depth','ar_AA_samples','ar_GI_specular_samples','ar_GI_transmission_samples',
                'ar_GI_diffuse_samples', 'ar_GI_specular_depth', 'ar_bucket_size']


raw_dataset = pd.read_csv("output_data.csv", usecols=column_names, sep=",")

dataset = raw_dataset.copy()

dataset = dataset.apply(pd.to_numeric, downcast='float') # convert all columns of DataFrame

dataset.tail()


,/render/frame time/rendering/microseconds,ar_GI_diffuse_depth,ar_AA_samples,ar_GI_specular_samples,ar_GI_transmission_samples,ar_GI_diffuse_samples,ar_bucket_size,ar_GI_specular_depth
983,1.531070e+09,3.0,4.0,2.0,10.0,9.0,277.0,6.0
984,3.426094e+09,8.0,10.0,7.0,4.0,9.0,97.0,7.0
985,2.747112e+08,6.0,7.0,9.0,8.0,5.0,619.0,9.0
986,2.204407e+09,10.0,5.0,8.0,1.0,8.0,979.0,3.0
987,4.320932e+08,9.0,10.0,6.0,9.0,10.0,883.0,3.0


In [3]:
dataset.isna().sum()

/render/frame time/rendering/microseconds    0
ar_GI_diffuse_depth                          0
ar_AA_samples                                0
ar_GI_specular_samples                       0
ar_GI_transmission_samples                   0
ar_GI_diffuse_samples                        0
ar_bucket_size                               0
ar_GI_specular_depth                         0
dtype: int64

In [4]:
dataset.tail()

,/render/frame time/rendering/microseconds,ar_GI_diffuse_depth,ar_AA_samples,ar_GI_specular_samples,ar_GI_transmission_samples,ar_GI_diffuse_samples,ar_bucket_size,ar_GI_specular_depth
983,1531069906,3.0,4.0,2.0,10.0,9.0,277.0,6.0
984,3426094115,8.0,10.0,7.0,4.0,9.0,97.0,7.0
985,274711210,6.0,7.0,9.0,8.0,5.0,619.0,9.0
986,2204407029,10.0,5.0,8.0,1.0,8.0,979.0,3.0
987,432093194,9.0,10.0,6.0,9.0,10.0,883.0,3.0


In [12]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_dataset.tail()
#test_dataset

,/render/frame time/rendering/microseconds,ar_GI_diffuse_depth,ar_AA_samples,ar_GI_specular_samples,ar_GI_transmission_samples,ar_GI_diffuse_samples,ar_bucket_size,ar_GI_specular_depth
464,5449300,2.0,8.0,5.0,1.0,7.0,848.0,4.0
25,55332392,10.0,9.0,6.0,4.0,0.0,732.0,0.0
110,246437990,6.0,8.0,5.0,5.0,2.0,917.0,6.0
149,435017108,9.0,3.0,5.0,5.0,9.0,322.0,2.0
152,9427974,10.0,4.0,3.0,0.0,2.0,220.0,0.0


In [6]:
#sns.pairplot(train_dataset[column_names], diag_kind="kde")

In [7]:
train_stats = train_dataset.describe()
train_stats.pop(column_names[0])
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
ar_GI_diffuse_depth,790.0,5.229114,3.192500,0.0,2.25,5.0,8.00,10.0
ar_AA_samples,790.0,5.553165,2.854136,1.0,3.00,6.0,8.00,10.0
ar_GI_specular_samples,790.0,4.894937,3.184301,0.0,2.00,5.0,8.00,10.0
ar_GI_transmission_samples,790.0,4.812658,3.257102,0.0,2.00,5.0,8.00,10.0
ar_GI_diffuse_samples,790.0,5.075949,3.034955,0.0,3.00,5.0,8.00,10.0
ar_bucket_size,790.0,517.620253,296.006920,16.0,274.00,509.5,778.75,1024.0
ar_GI_specular_depth,790.0,4.824051,3.127729,0.0,2.00,5.0,7.00,10.0


In [8]:
#test_dataset.tail()
train_labels = train_dataset.pop(column_names[0])
test_labels = test_dataset.pop(column_names[0])

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [9]:
def build_model():
    
  model = Sequential([
    tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                512       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 4,737
Trainable params: 4,737
Non-trainable params: 0
_________________________________________________________________


In [11]:
#normed_train_data[:10]

example_batch = normed_train_data[:2]
example_result = model.predict(example_batch)
#example_batch

KeyError: "None of [Int64Index([0, 1], dtype='int64')] are in the [columns]"